# 🧬 RNA-seq 분석 Pipeline 정리

## 1️⃣ 현재 디렉토리 확인
```bash
pwd
```

## 2️⃣ FASTQC 실행 (리드 품질 검사)
```bash
# FASTQC 결과 저장 폴더 생성
mkdir -p QC_reports/FastQC

# 각 샘플 폴더에서 FASTQC 실행
for dir in /common/weinera2lab/UCLA/PSMA/WTS/*; do
    fastqc -o QC_reports/FastQC "$dir"/*.fastq.gz
done

# 결과 확인
ls -lh QC_reports/FastQC/
```

## 3️⃣ MultiQC 실행 (FASTQC 결과 종합 리포트 생성)
```bash
# MultiQC 버전 확인
multiqc --version

# MultiQC 실행
multiqc -o /common/weinera2lab/UCLA/PSMA/WTS/QC_reports/MultiQC /common/weinera2lab/UCLA/PSMA/WTS/QC_reports/FastQC

# 결과 확인
ls -lh /common/weinera2lab/UCLA/PSMA/WTS/QC_reports/MultiQC/
```

## 4️⃣ Read Filtering (Adapter 제거 및 저품질 리드 필터링)
📌 **왜 필요한가?**  
- 시퀀싱 과정에서 남아있는 Adapter 서열을 제거하여 downstream 분석의 정확도를 높임  
- 저품질 염기 및 짧은 리드를 제거하여 깨끗한 데이터 생성  

```bash
nano run_fastp.sh
```

### run_fastp.sh 파일 내용:
```bash
#!/bin/bash

INPUT_DIR="/BIO4/JSeo/UCLA/PSMA/RNA"
OUTPUT_DIR="/BIO4/JSeo/UCLA/PSMA/RNA_trimmed"

mkdir -p "$OUTPUT_DIR"

for sample_dir in "$INPUT_DIR"/*; do
    if [ -d "$sample_dir" ]; then
        sample_name=$(basename "$sample_dir")
        R1_FILE=$(ls "$sample_dir"/*_R1_001.fastq.gz)
        R2_FILE=$(ls "$sample_dir"/*_R2_001.fastq.gz)

        OUTPUT_R1="$OUTPUT_DIR/${sample_name}_R1_trimmed.fastq.gz"
        OUTPUT_R2="$OUTPUT_DIR/${sample_name}_R2_trimmed.fastq.gz"

        echo "Processing $sample_name ..."

        fastp \
        -i "$R1_FILE" -I "$R2_FILE" \
        -o "$OUTPUT_R1" -O "$OUTPUT_R2" \
        -q 25 -l 30 --cut_right --cut_right_mean_quality 20 \
        --detect_adapter_for_pe --thread 8 \
        --json "$OUTPUT_DIR/${sample_name}_fastp_report.json" \
        --html "$OUTPUT_DIR/${sample_name}_fastp_report.html"

        echo "Completed: $sample_name"
    fi
done

echo "All samples processed successfully!"
```

```bash
# 실행 권한 부여
chmod +x run_fastp.sh

# 실행
./run_fastp.sh
```

## 5️⃣ STAR Genome Index 생성 (필수)
📌 **왜 필요한가?**  
- STAR을 사용해 리드 데이터를 Reference Genome에 정렬하기 위해 Index를 생성해야 함.

```bash
nano run_star_pipeline.sh
```

### run_star_pipeline.sh 파일 내용:
```bash
#!/bin/bash

REFERENCE_DIR="/BIO4/JSeo/UCLA/PSMA/reference"
STAR_INDEX_DIR="/BIO4/JSeo/UCLA/PSMA/STAR_index"
RNA_TRIMMED_DIR="/BIO4/JSeo/UCLA/PSMA/RNA_trimmed"
STAR_OUTPUT_DIR="/BIO4/JSeo/UCLA/PSMA/STAR_output"

mkdir -p "$REFERENCE_DIR" "$STAR_INDEX_DIR" "$STAR_OUTPUT_DIR"

# Reference Genome 다운로드
if [ ! -f "$REFERENCE_DIR/genome.fasta" ]; then
    echo "Downloading Reference Genome..."
    wget -O "$REFERENCE_DIR/genome.fasta.gz" ftp://ftp.ensembl.org/pub/release-110/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
    gunzip "$REFERENCE_DIR/genome.fasta.gz"
fi

# Annotation GTF 다운로드
if [ ! -f "$REFERENCE_DIR/annotation.gtf" ]; then
    echo "Downloading Annotation GTF..."
    wget -O "$REFERENCE_DIR/annotation.gtf.gz" ftp://ftp.ensembl.org/pub/release-110/gtf/homo_sapiens/Homo_sapiens.GRCh38.110.gtf.gz
    gunzip "$REFERENCE_DIR/annotation.gtf.gz"
fi

# STAR Genome Index 생성
if [ ! -f "$STAR_INDEX_DIR/Genome" ]; then
    echo "Generating STAR Genome Index..."
    STAR --runThreadN 8 \
        --runMode genomeGenerate \
        --genomeDir "$STAR_INDEX_DIR" \
        --genomeFastaFiles "$REFERENCE_DIR/genome.fasta" \
        --sjdbGTFfile "$REFERENCE_DIR/annotation.gtf" \
        --sjdbOverhang 100
fi

echo "All steps completed successfully!"
```

```bash
# 실행 권한 부여
chmod +x run_star_pipeline.sh

# 실행
./run_star_pipeline.sh
```

## 6️⃣ FeatureCounts 실행 (유전자 발현량 정량화)
📌 **왜 필요한가?**  
- 정렬된 BAM 파일을 기반으로 각 유전자에 매핑된 read 수를 계산해야 함.

```bash
# FeatureCounts 실행
featureCounts -T 8 \
    -a /BIO4/JSeo/UCLA/PSMA/reference/annotation.gtf \
    -o /BIO4/JSeo/UCLA/PSMA/RNA/RNA_FeatureCounts/gene_counts.txt \
    /BIO4/JSeo/UCLA/PSMA/RNA/RNA_STAR_Results/STAR_output/*.bam
```

```bash
# MultiQC 실행 (FeatureCounts 결과 확인)
multiqc /BIO4/JSeo/UCLA/PSMA/RNA/RNA_FeatureCounts/
```
